# ETL Project Name: Boston Airbnb market analysis
Primary independent variable: Price of the listing 
Other dependent variables: availability, review, location, host since, response time, host acceptance rate, no. of host listing
Plots 
Prices vs review (How to categorize a review as Good, Bad and Neutral?)
Price vs host location
Price vs host since
Price vs host response time
Price vs host acceptance rate
Price vs No. of host listing


## Import Packages and Libraries

In [131]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine
#from ET_mysql_key import ET_key
from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"



### Import csv files

In [141]:
# File to Load

# 1.) Listing file had information of each listing in Boston area. 
#There are 95 columns with listing info, space and features, pictures, availability, price, host reviews, policy etc. 
listing_data_to_load = "data/listings.csv"

# 2.) Calender file has availibility of rentals. It has listing_id, date, available (t or f) and price
availability_data_to_load = "data/calendars.csv"

# 3.) Review file contanin information such as listing_id, id, date, reviewer_id, reviewer_name & comments
review_data_to_load = "data/reviews.csv"


## EXTRACTION

In [142]:
# Reading files and then storing into Pandas data frame. # Create DataFrames
# used pd.read to read & give out DataFrame(or TextParser) for the variables assigned to the pandas DateFrame are
#availability_data_to_load & listing_data_to_load & review_data_to_load  

In [143]:
# Listing data
# Show number of row and columns.# Display the data table for preview. Used head() to get the three rows
listing_df = pd.read_csv(listing_data_to_load)
listing_df.shape
pd.set_option('display.max_rows', 3)
pd.set_option('display.max_columns', 10)
pd.set_option('display.width', 100)
pd.set_option('precision', 5)
listing_df.style.highlight_null(null_color='red')
#lising_df.style
listing_df.head(3)


,listing_id,listing_url,scrape_id,last_scraped,name,...,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,2.01609e+13,9/7/2016,Sunny Bungalow in the City,...,moderate,f,f,1,NaN
1,3075044,https://www.airbnb.com/rooms/3075044,2.01609e+13,9/7/2016,Charming room in pet friendly apt,...,moderate,f,f,1,1.30
2,6976,https://www.airbnb.com/rooms/6976,2.01609e+13,9/7/2016,Mexican Folk Art Haven in Boston,...,moderate,t,f,1,0.47


In [145]:
# Availability data.
# Show number of row and columns.# Display the data table for preview. Used head() to get the three rows
availability_df = pd.read_csv(availability_data_to_load)
availability_df.shape
#pd.set_option('display.max_rows', 3)
#pd.set_option('display.max_columns', 5)
#pd.set_option('display.width', 100)
pd.set_option('precision', 5)
listing_df.style.highlight_null(null_color='red')
availability_df.head(3)

,listing_id,date,available,price
0,12147973,9/5/2017,f,NaN
1,12147973,9/4/2017,f,NaN
2,12147973,9/3/2017,f,NaN


In [146]:
# Review data
# Show number of row and columns.Display the data table for preview. Used head() to get the three rows
review_df = pd.read_csv(review_data_to_load)
review_df.shape
pd.option_context('display.colheader_justify','left')
review_df.style.set_properties(**{'text-align': 'left'})
review_df.head(3)

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,1178162,4724140,5/21/2013,4298113,Olivier,My stay at islam's place was really cool! Good...
1,1178162,4869189,5/29/2013,6452964,Charlotte,Great location for both airport and city - gre...
2,1178162,5003196,6/6/2013,6449554,Sebastian,We really enjoyed our stay at Islams house. Fr...


## TRANSFORM: Cleaning and Merging 

In [139]:
# Listing data.
# Show number of row and columns.
# Show stats.
# Removing Replace NaN with zero. unnecssary columns, standardizing column names, and creating a new df

In [147]:
#listing_df = listing_df[["listing_id","minimum_nights","host_response_time","host_acceptance_rate","price"]]
listing_df

,listing_id,listing_url,scrape_id,last_scraped,name,...,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,12147973,https://www.airbnb.com/rooms/12147973,2.01609e+13,9/7/2016,Sunny Bungalow in the City,...,moderate,f,f,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...
3584,14504422,https://www.airbnb.com/rooms/14504422,2.01609e+13,9/7/2016,(K1) Private Room near Harvard/MIT,...,flexible,f,f,3,NaN


In [148]:
#merge_table.head(5)
# Combine the data into a single dataset
    #used the pd.merge merge DataFrame city_pd and ride_pd based on the common list city 
    #ALso, used how left based on result table needed and also specified how as left
    #Used head() to get the five rows
mergeal_table = pd.merge(availability_df,listing_df,how="outer",on=["listing_id","listing_id"])
mergealr_table = pd.merge(mergeal_table,review_df,how="outer",on=["listing_id","listing_id"])
# Display the data table for preview
mergealr_table.head(5)

,listing_id,date_x,available,price_x,listing_url,...,id,date_y,reviewer_id,reviewer_name,comments
0,12147973,9/5/2017,f,NaN,https://www.airbnb.com/rooms/12147973,...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4,12147973,9/1/2017,f,NaN,https://www.airbnb.com/rooms/12147973,...,NaN,NaN,NaN,NaN,NaN


In [149]:

mergealr_table.describe()

,listing_id,scrape_id,host_id,host_listings_count,host_total_listings_count,...,jurisdiction_names,calculated_host_listings_count,reviews_per_month,id,reviewer_id
count,21044952.0,2.10450e+07,21044952.0,21044952.0,21044952.0,...,0.0,21044952.0,2.08251e+07,20825067.0,20825067.0
...,...,...,...,...,...,...,...,...,...,...,...
max,14933461.0,2.01609e+13,93854106.0,749.0,749.0,...,NaN,136.0,1.91500e+01,99990454.0,93350341.0


In [150]:
#mergealr_table.set_option('display.expand_frame_repr', False)
#mergealr_table.columns
mergealr_table.columns.tolist()

['listing_id',
 'date_x',
 'available',
 'price_x',
 'listing_url',
 'scrape_id',
 'last_scraped',
 'name',
 'summary',
 'space',
 'description',
 'experiences_offered',
 'neighborhood_overview',
 'notes',
 'transit',
 'access',
 'interaction',
 'house_rules',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
 'host_about',
 'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood',
 'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'be

In [151]:
mergealr_table["price_x"] = mergealr_table["price_x"].replace(r'^\s+$', 208, regex=True)

In [152]:
mergealr_table["price_x"].tolist()

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [76]:
mergealr_table["price_x"]

0           NaN
           ... 
25196679    NaN
Name: price_x, Length: 25196680, dtype: object

In [77]:
mergealr_table['price_x'] = mergealr_table['price_x'].fillna(209)

In [57]:
listing_df.shape
listing_df.describe()
#listing_df.style.highlight_null(null_color='red')
listing_df1=listing_df.drop(['experiences_offered','host_about',
 'scrape_id',
 'last_scraped',
  'space',
 'description',
  'transit',
 'access',
 'interaction',
 'house_rules',
  'host_id',
 'host_url',
 'host_name',
 'host_since',
 'host_location',
  'host_response_time',
 'host_response_rate',
 'host_acceptance_rate',
 'host_is_superhost',
  'host_listings_count',
 'host_total_listings_count',
 'host_verifications',
 'host_has_profile_pic',
 'host_identity_verified',
 'street',
 'neighbourhood',
 'neighbourhood_cleansed',
 'neighbourhood_group_cleansed',
 'city',
 'state',
 'zipcode',
 'market',
 'smart_location',
 'country_code',
 'country',
 'latitude',
 'longitude',
 'is_location_exact',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'bed_type',
 'amenities',
 'square_feet',
 'price',
 'weekly_price',
 'monthly_price',
 'security_deposit',
 'cleaning_fee',
 'guests_included',
 'extra_people',
 'minimum_nights',
 'maximum_nights',
 'calendar_updated',
 'has_availability',
 'availability_30',
 'availability_60',
 'availability_90',
 'availability_365',
 'calendar_last_scraped',
 'number_of_reviews',
 'first_review',
 'last_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'requires_license',
 'license',
 'jurisdiction_names',
 'instant_bookable',
 'cancellation_policy',
 'require_guest_profile_picture',
 'require_guest_phone_verification',
 'calculated_host_listings_count',
 'reviews_per_month'],axis=1)
listing_df1.columns.tolist()

['listing_id',
 'listing_url',
 'name',
 'summary',
 'neighborhood_overview',
 'notes',
 'thumbnail_url',
 'medium_url',
 'picture_url',
 'xl_picture_url',
 'host_thumbnail_url',
 'host_picture_url',
 'host_neighbourhood']

In [111]:
mergealr_table['price_x'].tolist()

['price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'pr

In [79]:
mergealr_table['latitude']

0           42.28262
              ...   
25196679    42.38789
Name: latitude, Length: 25196680, dtype: float64

In [116]:

#price=mergealr_table['price_x']
#latitude=mergealr_table['latitude']
#plt.scatter(latitude, price)
plt.show()

In [117]:
mergealr_table["price_x"].tolist()

['price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'price_x',
 'pr

In [118]:
host_location=mergealr_table['host_location']
host_location

0               Boston, Massachusetts, United States
                              ...                   
25196679    Somerville, Massachusetts, United States
Name: host_location, Length: 25196680, dtype: object

In [69]:
##coordinates for plotting 

long_max = df['longitude'].max() + .02
long_min = df['longitude'].min() -.02
mid_long = (df['longitude'].min() + df['longitude'].max())/2

lat_max = df['latitude'].max() + .02
lat_min = df['latitude'].min() - .02
mid_lat = (df['latitude'].min() + df['latitude'].max())/2

## map
m = Basemap(projection='cyl',lat_0=mid_lat,lon_0=mid_long,\
            llcrnrlat=lat_min,urcrnrlat=lat_max,\
            llcrnrlon=long_min,urcrnrlon=long_max,\
            rsphere=6371200.,resolution='h',area_thresh=10)
m.drawcoastlines()
m.drawstates()
m.drawcounties()
m.shadedrelief()

## locations
x, y = m(df2['longitude'], df2['latitude'])
sp = plt.scatter(x, y, c=df2['price'], s=15)



plt.rcParams["figure.figsize"] = [11,7]
cb = plt.colorbar(sp)
cb.set_label('Price($)')
plt.show()
plt.clf()

NameError: name 'df' is not defined